In [23]:
import os
import sys

import dvc
import pandas as pd
import numpy as np

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier

In [47]:
DATA_PATH = os.path.join(os.path.abspath("../.."), "data/")

In [42]:
# get dataset
wine = load_wine()

# load dataframe
data = pd.DataFrame(data= np.c_[wine['data'], wine['target']],
                     columns= wine['feature_names'] + ['target'])

# save data locally
data.to_parquet(DATA_PATH + "data.parquet")

# DVC How - To

1. dvc init to initialize 
2. dvc add <folder> to start tracking
   


In [46]:
data.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0.0


In [48]:
data = data.drop(["ash", "magnesium", "flavanoids"], axis=1)

In [49]:
data.to_parquet(DATA_PATH + "data.parquet")

In [50]:
data.shape

(178, 11)

In [51]:
data_restored = pd.read_parquet(DATA_PATH + "data.parquet")

In [52]:
data_restored.shape

(178, 14)

In [55]:
data_restore_dropped = pd.read_parquet(DATA_PATH + "data.parquet")

In [56]:
data_restore_dropped.shape

(178, 11)